In [1]:
import numpy as np
import matplotlib.pyplot as plt

import histlite as hl
import csky as cy
import glob as glob

%matplotlib inline

In [2]:
timer = cy.timing.Timer()
time = timer.time

In [27]:
#fake GRL 
current_mjd = 60369
runstart = 135328
run_duration = 1/3 #8h
nr_runs = 2000
background_event_rate = 4.587

new_grl_entries = []
dtype = [('run', int), ('start', np.float64), ('stop', np.float64), ('livetime', np.float64), ('events', int)]
for i in range(runstart, runstart+nr_runs+1):
    new_grl_entries.append((i, current_mjd, current_mjd + run_duration, run_duration, background_event_rate))
    current_mjd += run_duration

grl = np.array(new_grl_entries, dtype=dtype)

- We used the GettingStarted (https://docs.icecube.aq/csky/main/01_getting_started.html) and Implementation Inspection tutorial (https://docs.icecube.aq/csky/main/06_implementation_inspection.html) from csky as a guide 
- Also: we looked at the Greco GRB 2022 analysis (https://github.com/icecube/wg-nu-sources/blob/main/2022_GRECO_GRB/scripts/01_get_no_prior_background_allsky_scan.py)
- We tried to make our own dataset

# Try to make our own dataset
- We use the following queso data: data/sim/IceCubeUpgrade/genie/level4_queso/ (120028, 121028, 140028, 141028, 160028, 161028) & 
data/sim/IceCubeUpgrade/muongun/level4_queso/130028
- using Michael’s .i3 to .npy converter: https://github.com/mjlarson/nusources_dataset_converters/tree/main
- 


In [30]:
# Reading in the signal and background files and change the keys to the correct ones --> so that csky can read it

def get_signal_files(key):
    dir = "/data/user/bschlueter/software/nusources_dataset_converters/data/queso_"+str(key)+".npy"
    sig = np.load(dir)
    sig = np.hstack(sig)
    sig_upgraded = np.lib.recfunctions.rename_fields(sig, {'ow': 'oneweight','trueE':'true_energy'})
    sig_upgraded = sig_upgraded.view(np.recarray)
    return sig_upgraded

bg_filename = "/data/user/bschlueter/software/nusources_dataset_converters/data/muongun_131028.npy"
bg = np.load(bg_filename)
bg_upgraded = np.lib.recfunctions.rename_fields(bg, {'atmo_weight': 'weight'})
bg_upgraded = bg_upgraded.view(np.recarray)

In [33]:
# try to make the own dataset 

keys = ['120028','121028','140028','141028','160028','161028']
Dataset = []
for key in keys:  
    Dataset.append(cy.selections.Dataset(key = key,livetime=8*60*60, sig= get_signal_files(key), bg = bg_upgraded))

#and then create an ana as in the tutorial
with time('ana setup (from scratch)'):
    ana = cy.get_analysis(cy.selections.repo,spec = Dataset)

Setting up Analysis for:

Done.

0:00:00.000642 elapsed.


In [34]:
Dataset

[Dataset('120028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.60,
 Dataset('121028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.41,
 Dataset('140028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.46,
 Dataset('141028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.47,
 Dataset('160028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.02,
 Dataset('161028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.50]

<span style="color:red;">Problem: empty keys in ana.</span>

In [35]:
cy.CONF['ana'] = ana
cy.CONF['mp_cpus'] = 1
print(ana)

Analysis(keys=[])
